# Portafolio de Crypto

Acabo de meter mi dinero a Binance, estaba haciendo copy-traiding, pero los que le mueven no saben de esto :p

### Código

In [1]:
import pandas as pd
import yfinance as yf

from datetime import datetime

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [2]:
cryptos = [
    "BTC-USD",  # Bitcoin
    "ETH-USD",  # Ethereum
    "USDT-USD", # Tether
    "BNB-USD",  # Binance Coin
    "USDC-USD", # USD Coin
    "XRP-USD",  # XRP
    "ADA-USD",  # Cardano
    "SOL-USD",  # Solana
    "DOGE-USD", # Dogecoin
    "TRX-USD",  # Tron
    "TON-USD",  # Toncoin
    "MATIC-USD",# Polygon
    "DOT-USD",  # Polkadot
    "LTC-USD",  # Litecoin
    "AVAX-USD", # Avalanche
    "WBTC-USD", # Wrapped Bitcoin
    "DAI-USD"   # DAI
]

In [3]:
adj_close_list = []

for crypto in cryptos:
    
    stock = yf.Ticker(crypto)
    hist = stock.history(period="max")
    adj_close_list.append(hist['Close'].rename(crypto))

    data = pd.concat(adj_close_list, axis=1)

    data.to_csv("crypto_adj_close.csv", index=True)

    data.dropna(inplace=True) 

In [4]:
def portafolio(data, total_investment: int, obj: str, start_date: str, end_date: str, date: bool, freq: int):

    if date:
        data = data.loc[start_date:end_date]

    mu = expected_returns.mean_historical_return(data, frequency=freq)
    s = risk_models.sample_cov(data, frequency=freq)

    ef = EfficientFrontier(mu, s)
    
    if obj == "max sharpe":
        weights = ef.max_sharpe()

    elif obj == "min volatuility":
        weights = ef.min_volatility()

    elif obj == "max return":
        weights = ef.max_quadratic_utility()

    elif obj == "efficient_risk":
        weights = ef.efficient_risk(0.1)

    elif obj == "efficient_return":
        weights = ef.efficient_return(0.1)

    amounts = ef.clean_weights()

    for crypto, weight in weights.items():
        amounts[crypto] = weight * total_investment

    print(f"Amounts to invest in each crypto to {obj}:")
    for crypto, amount in amounts.items():
        if amount > 0:
            print(f"{crypto}: ${amount:.2f}")

    print("\nExpected return, volatility and Sharpe ratio:")

    print(ef.portfolio_performance(verbose=True))

    return

### Ejemplos

In [5]:
portafolio(data, 1454, "max sharpe", "2023-12-14", "2024-12-12", date=True, freq=365)

Amounts to invest in each crypto to max sharpe:
BNB-USD: $490.24
XRP-USD: $525.02
DOGE-USD: $276.92
TRX-USD: $161.82

Expected return, volatility and Sharpe ratio:
Expected annual return: 238.8%
Annual volatility: 57.5%
Sharpe Ratio: 4.12
(2.3879676633084648, 0.5749177206231751, 4.118794008891802)


In [6]:
portafolio(data, 1000, "max sharpe", "2023-12-14", "2024-11-12", date=True, freq=15)

Amounts to invest in each crypto to max sharpe:
BNB-USD: $504.50
SOL-USD: $63.71
DOGE-USD: $431.79

Expected return, volatility and Sharpe ratio:
Expected annual return: 5.1%
Annual volatility: 13.8%
Sharpe Ratio: 0.23
(0.05112196132210648, 0.13816164100832815, 0.22525761198964409)
